## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-15-13-00-19 +0000,bbc,Flash floods kill more than 160 in Pakistan,https://www.bbc.com/news/articles/cn02g0n2kepo...
1,2025-08-15-13-00-00 +0000,missionlocal,Warm embrace,https://missionlocal.org/2025/08/sf-photos-war...
2,2025-08-15-12-59-58 +0000,nyt,Rosie O’Donnell and Eating the Rich: 7 Buzzy S...,https://www.nytimes.com/2025/08/15/arts/edinbu...
3,2025-08-15-12-57-59 +0000,nyt,United Nations Plastic Treaty Talks End Withou...,https://www.nytimes.com/2025/08/15/climate/pla...
4,2025-08-15-12-55-32 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/08/15/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2308/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
27,trump,56
28,putin,24
105,will,16
60,ukraine,12
59,war,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
69,2025-08-15-09-03-29 +0000,nyt,"How Trump’s Meeting With Putin Could Unfold, a...",https://www.nytimes.com/2025/08/15/us/politics...,163
141,2025-08-15-01-00-00 +0000,wsj,"President Trump, by meeting Vladimir Putin on ...",https://www.wsj.com/politics/national-security...,144
99,2025-08-15-06-37-53 +0000,cbc,Trump tries to turn the page on Ukraine war wi...,https://www.cbc.ca/news/politics/trump-putin-r...,131
95,2025-08-15-07-00-55 +0000,wapo,"How Putin, ex-KGB officer, will seek to sway T...",https://www.washingtonpost.com/world/2025/08/1...,129
112,2025-08-15-04-27-14 +0000,nypost,AG Pam Bondi declares DC ‘will not remain a sa...,https://nypost.com/2025/08/15/us-news/ag-pam-b...,126


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
69,163,2025-08-15-09-03-29 +0000,nyt,"How Trump’s Meeting With Putin Could Unfold, a...",https://www.nytimes.com/2025/08/15/us/politics...
112,70,2025-08-15-04-27-14 +0000,nypost,AG Pam Bondi declares DC ‘will not remain a sa...,https://nypost.com/2025/08/15/us-news/ag-pam-b...
178,59,2025-08-14-22-26-00 +0000,wsj,California Democrats are planning their own re...,https://www.wsj.com/politics/policy/gavin-news...
288,44,2025-08-14-15-59-01 +0000,nypost,Trump reveals the odds of Putin summit ending ...,https://nypost.com/2025/08/14/us-news/trump-gi...
60,44,2025-08-15-09-30-00 +0000,wsj,Tariffs haven’t caused inflation to surge as m...,https://www.wsj.com/economy/trade/trump-tariff...
301,42,2025-08-14-14-49-42 +0000,cbc,Former justice minister Irwin Cotler calls on ...,https://www.cbc.ca/news/canada/montreal/irwin-...
162,34,2025-08-14-23-20-57 +0000,nyt,"In L.A., Fear of ICE Raids Put the First Day o...",https://www.nytimes.com/2025/08/14/us/los-ange...
229,33,2025-08-14-19-55-56 +0000,nyt,Air Canada Cancels Flights Ahead of Possible F...,https://www.nytimes.com/2025/08/14/world/canad...
235,33,2025-08-14-19-04-00 +0000,wsj,U.N. Report Finds Widespread Killing by Syrian...,https://www.wsj.com/world/middle-east/u-n-repo...
142,31,2025-08-15-01-00-00 +0000,wsj,The nine law firms that promised President Tru...,https://www.wsj.com/politics/policy/trump-pro-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
